# Open Source vc OpenAI

Did GPT-4 get worse? Is Llama 2 a better model? Run this notebook to find out.

We'll use auto-evaluation by GPT-4 to measure outputs from Llama 2, as well as gpt-4 (current and frozen versions) across a few prompts. To make this example easy to run, we'll be using a 7B GGML variant of the Llama model. This should be able to run on a typical laptop.

## Installations

You can setup prompttools either by installing via `pip` or using `python setup.py develop` in the root of this repo. Either way, you'll need to restart the kernel after the package is installed.

In [ ]:
# !pip install --quiet --force-reinstall prompttools

## Setup imports and API keys

Next, we'll need to set our API keys. Since we want to use GPT-4 for auto-eval, we need to set that one.

In [18]:
import os
os.environ['DEBUG'] = ""
os.environ['OPENAI_API_KEY'] = ""

Then we'll import the relevant `prompttools` modules to setup our experiment.

In [19]:
from typing import Dict, List, Tuple
from prompttools.experiment import LlamaCppExperiment
from prompttools.experiment import OpenAIChatExperiment
from prompttools.harness.multi_experiment_harness import MultiExperimentHarness
from prompttools.selector.prompt_selector import PromptSelector

## Run an experiment

To set up this experiment, we need to use a `PromptSelector`. This is because the input formats for Llama 2 and GPT-4 are different. While GPT-4 is run with a chat history, Llama2 takes text input. A `PromptSelector` allows us to pass the same prompt to different models, and render the necessary object at request time.

In [ ]:
instructions = ["""
You are a sales development representative for a startup called Hegel AI.
Your startup builds developer tools for large language models.
""",
"""
You are a customer support representative for a startup called Hegel AI.
Answer the following customer question:
""",         
"""
You are a helpful math tutor.
Answer the following math problem:
"""]
inputs = ["""
Draft a short sales email, 50 words or less, asking a prospect for 15 minutes
of their time to chat about how they're using large language models.
""",
"""
Do you offer refunds?
""",
"""
Is 7 a prime number?
"""]
selectors = [PromptSelector(instructions[i], inputs[i]) for i in range(3)]

Next, we create our test inputs. We can iterate over models, inputs, and configurations like temperature.

In [ ]:
model_paths = ['/your/path/to/llama-2-7b-chat.ggmlv3.q2_K.bin']  # Download from https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGML/tree/main
temperatures = [1.0]
call_params = dict(temperature=temperatures)
llama_experiment = LlamaCppExperiment(model_paths, selectors, call_params=call_params)

In [ ]:
models = ['gpt-4-0314', 'gpt-4-0613', 'gpt-4']
temperatures = [0.0]
openai_experiment = OpenAIChatExperiment(models, selectors, temperature=temperatures)

After that - we define our harness to run experiments

In [ ]:
harness = MultiExperimentHarness([openai_experiment, llama_experiment])

In [ ]:
harness.prepare()
harness.run()

Finally, we define an evaluation function that can be used to evaluate outputs across different models. Notice that the extract resp

In [ ]:
from prompttools.utils import autoeval


def extract_responses(output) -> str:
    if "text" in output["choices"][0]:
        return [choice["text"] for choice in output["choices"]]
    else:
        return [choice["message"]["content"] for choice in output["choices"]]


def use_gpt4(
    prompt: str, results: Dict, metadata: Dict
) -> float:
    """
    A simple test that checks semantic similarity between the user input
    and the model's text responses.
    """
    distances = [
        autoeval.compute(prompt, response)
        for response in extract_responses(results)
    ]
    return min(distances)


Finally, we can evaluate and visualize the results.

In [ ]:
harness.evaluate("auto-evaluation", use_gpt4)

In [ ]:
harness.visualize()

In [ ]:
harness.visualize("response(s)")